<h1>Сверка с банками и digital партнёрами<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Инструкция" data-toc-modified-id="Инструкция-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Инструкция</a></span></li><li><span><a href="#Сверка-реестра-для-подтверждения-контрагенту" data-toc-modified-id="Сверка-реестра-для-подтверждения-контрагенту-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Сверка реестра для подтверждения контрагенту</a></span></li><li><span><a href="#Подготовка-реестра-для-заливки-в-отчет-по-прибыли-по-продлениям" data-toc-modified-id="Подготовка-реестра-для-заливки-в-отчет-по-прибыли-по-продлениям-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка реестра для заливки в отчет по прибыли по продлениям</a></span></li><li><span><a href="#Подготовка-реестра-для-заливки-в-отчет-по-прибыли-от-кэшбэка" data-toc-modified-id="Подготовка-реестра-для-заливки-в-отчет-по-прибыли-от-кэшбэка-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Подготовка реестра для заливки в отчет по прибыли от кэшбэка</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Инструкция
  1. Код написан 07.2021 на python 3. Для работы файла необходимо, чтобы все библиотеки и модули из ячейки номер 1 были установлены. Включить VPN-соединение. У пользователя, от имени которого запускается данный файл, наверное должен быть доступ к БД Vertica, хотя данный код не подразумевает ввода какого-либо пароля.
  2. Исходный для сверки файл Excel, полученный от партнёра необходимо положить в ту же папку, где лежит sverka_backend.ipynb (если вы читаете данный текст, значит, файл sverka_backend.ipynb вы уже положили в какую-то папку). 
  3. Переобозвать исходный файл в input.xlsx. 
  4. Открыть input.xlsx и изменить названия на **выделенные** у следующих столбцов:
    - **date** дата транзакции (формат обязательно дд.мм.гггг). Может быть датой или текстом - home credit умудряется присылать реестры, где смешаны и те, и те типы данных в столбце с датами.
    - **sum** сумма тарнзакции в руб, включая НДС (числовой формат)
    - **code** артикул партнёра (можно выбрать любой столбец - дальше он никак не участвует, строчный формат)
    - **card** маска банковской карты (строчный формат)
    - **rrn** RRN (Reference Retrieval Number) – это уникальный идентификатор банковской транзакции, который назначается банком Эквайрером при инициализации платежа. Должен быть 12-ти значным (бывает партнёры путают местами названия столбцов в своих выгрузках, бывает нули в начале RRN удаляются, если RRN сохранены в формате чисел, а не текста, то есть лучше проверить, что формат текстовый и все нули на месте). Например, БСП присылает почему-то 13-тизначный RRN - остаётся только по номеру карты сверять. Также необходимо проверить отсутствие или удалить внизу таблицы строки типа "Итого..." и т.д., отличающиеся от формата основной таблицы. Если их оставить, код будет падать.
    ХКФ и Озон присылают файлы Эксель со скрытыми листами, программа отрабатывает, если все скрытые листы пустые. Должен остаться один непустой лист.
  5. Выполняем в меню сверху Kernel/Restart&Run all/Restart and Run all Cells. Ядро перезагрузится и выполнит все ячейки кода по порядку, начиная с первой.
  6. С клавиатуры ввести название банка (скопировать и вставить из предложенного списка - это важно дальше для корректного отображения выручки по кэшбэкам в отчёте табло). Нажать Enter на клавиатуре.  
  Если название будет не из списка (пробел добавлен, или по-русски одна буква), то код упадёт и предложит перезапустить программу.
  8. Дождаться выполнения кода до конца (зависит от размеров файлов и загруженности БД - примерно пару минут) - появление в последней ячейке надписи на красном фоне: ["Выполнение программы завершено успешно"](https://disk.yandex.ru/i/VQo2Ai4kKnjyPg).
  9. В папке, где лежит исходный файл, открываем для проверки файл sverka.xlsx. Там две вкладки. Как правило принимаем сверку, если отклонение меньше 5% или 100тыс руб по сумме транзакций (выбираем по меньшему, то есть если расхождение 3% и 450тыс руб, то принимаем, и если 22% и 30тыс руб, то тоже принимаем). Если отклонение больше обозначенных, то сообщаем об этом менеджеру/делопроизводителю для дальнейших выяснений с партнёром. Сверка выполняется в следующем порядке: сначала сверяем по RRN, затем по дате,сумме (может отличаться), маске карты. Если по обеим схемам не находим транзакцию в Vertica, то незасчитываем данную транзакцию. 
  10. В папке, где лежит исходный файл, находится файл reestr_prodleniya.csv, полностью подготовленный для ручной заливки в таблицу akoblov.banks_cashback_prolong в ClickHouse, на которой держится отчёт
<code>[Выручка от продлений от покупок из реестров партнеров по кэшбэкам](https://tablo.ivi.ru/#/views/_569/sheet0?:iid=1)  
</code>
(Боже, храни человека, который придумал это название!).  
  11. Дальше необходимо reestr_prodleniya.csv (можно переименовать) импортировать, клацнув правой кнопкой мыши по названию таблицы akoblov.banks_cashback_prolong  
 <code>[Скрин с DBeaver](https://disk.yandex.ru/i/UAh7BRR-ta1HwQ)
</code>  
  12. На этапе импорта возможны выпадения ошибок из-за несоотвествия типов данных в заливаемом файле и таблице, или различии в разделителе. Перед заливкой обязательно убедиться, что типы данных и названия колонок бьются
 <code>[Проверка соответствия названий колонок и типов данных](https://disk.yandex.ru/i/E91fpVinRJWwtQ)
</code> 
  13. Для обновления отчёта в Табло необходимо выполнить  скрипт по шагам (три шага: 1.удаление таблицы ekapitanova.banks_prolong_copy 2.создание пустой таблицы 3.заполнение рассчётными данными на основании данных из таблицы с залитыми реестрами akoblov.banks_cashback_prolong - другого способа добавления данных без удаления для CH не знаю)
<code>[Скрипт](https://disk.yandex.ru/d/0WJaAh4xZ2cSjA)  
</code>  
Целесообразно данный пункт выполнять, когда по всем партнёрам будут залиты реестры за отчётный месяц в akoblov.banks_cashback_prolong по п.10 данной инструкции.
  14. Аналогично п.11-12 заливаем реестр по кэшбэкам reestr_cashback.csv в табличку epimenova.banks_cashback_revenue_registry_copy. На этой табличке уже висит отчет в Табло 
  <code>[Выручка по кэшбэкам через Банки-партнеры по реестрам партнеров. ](https://tablo.ivi.ru/#/views/_502/sheet0?:iid=1)  
</code>
  222. В процессе выполнения данного кода возможны выпадения ошибок.  
  Например: разрыв соеднинения с БД Vertica или её перегрузкой, тогда рекомендуется попробовать исполнить код в другое время, менее загруженное для БД.  
  Или некорректная предобработка исходного файла - см.п.2-4 настоящей Инструкции.  
  Также рекомендуется скачать в надёжное место все файлы по ссылкам в данной инструкции, потому что неизвестно как долго проживёт Яндекс.Диск.

# Сверка реестра для подтверждения контрагенту

In [1]:
import pandas as pd

import numpy as np

import xlsxwriter

import re

from datetime import datetime, timedelta, date

from dateutil.relativedelta import relativedelta

import vertica_python

#отключаем предупреждения 
import warnings
warnings.filterwarnings('ignore')

Загружаем необходимые библиотеки и отключаем предупреждения.

In [2]:
# функция для получения информации о датафрейме df
def info_df(df, see_rows=5):
    name =[x for x in globals() if globals()[x] is df][0] #получаем имя датафрейма

    print ('Датафрейм '+name)
    display(df.head(see_rows))

    display(df.describe().T)

    print ('Процент пропусков по столбцам')
    display((round((df.isna().sum()/len(df))*100,1)).astype(str) + '%')
    
    df.info()
    print (50*'x','\n')
    print ('')
 
# функция преобразования даты из строки в "правильный" формат дд.мм.гггг
def date_true(row):
    if '-' in row['date']:
        return row['date'][8:10]+'.'+row['date'][5:7]+'.'+row['date'][:4]
    else:
        return row['date']

# функция последнего дня месяца
def last_day_of_month(date):
    if date.month == 12:
        return date.replace(day=31)
    return date.replace(month=date.month+1, day=1) - timedelta(days=1)

# функция первого дня месяца
def first_day_of_month(date):
    return date.replace(day=1)

Прописываем все функции в начала, чтобы их легче искать.

In [ ]:
bank_name=str(input("""Введите название банка по-английски:
home
ozon
sovkom
spb
sweetcard
mkb
otkrytie
alfa
tinkoff
raiff\n\n"""
))

#задаём название колонок, которые будем открывать
cols = ['date', 'sum', 'code','card','rrn']

if bank_name in ['home',
                'ozon',
                'sovkom',
                'spb',
                'sweetcard',
                'mkb',
                'otkrytie',
                'alfa',
                'tinkoff',
                'raiff',
                ]:
    
    data = pd.read_excel(
                   'input.xlsx', 
                    usecols=cols, # загружаем только нужные колонки
                    sheet_name = None, # загружаем все листы, включая скрытые
                    dtype={
                        'sum': float,
                           'date': str, 'rrn': str,'card': str,'rrn': str,'code': str}
                        )
    
    # перебираем все листы загруженные листы и оставляем последний непустой 
    for i in data:
        if data[i].empty:
            print("\033[37m\033[41m\033{}\033[0m".format('В файле input.xlsx есть скрытые пустые листы'))
        else: 
            df = data[i]
else:
     print("\033[37m\033[41m\033{}\033[0m".format('Неверно введено название банка. Перезапустите программу с начала.')) 
    
info_df(df)

Введите название банка по-английски:
home
ozon
sovkom
spb
sweetcard
mkb
otkrytie
alfa
tinkoff
raiff

ozon


Вводим название банка и отчётный месяц.  
Октрываем файл с исходными данными.  
Смотрим инфу по исходному файлу.

In [ ]:
print ('Количество полных дубликатов: '+df.duplicated().sum().astype(str)+' строк')

# некоторые банки присылают в реестрах отрицательные суммы, что нам неинтересно
print ('Количество возвратов и нулевых транзакций: '+df['sum'].loc[df['sum']<=0].count().astype(str)+' строк')

sverka = df.drop_duplicates().query('sum>=0')

Удаляем полные дубликаты  строк по всем столбцам и нулевые транзакции при наличии.

In [ ]:
sverka.insert(0, 'bank_name', bank_name)
display(sverka.sample(1))

Вставляем колонку с названием банка в первый столбец.

In [ ]:
# если есть пропуски в датах или левые строчки типа Итого, удаляем
sverka = sverka.loc[~sverka['date'].isna()]

# выбираем дату из строки (может быть в двух форматах)
sverka['date']=sverka['date'].apply(lambda x: re.findall('^\d{2}.\d{2}.\d{4}|\d{4}-\d{2}-\d{2}',x)[0])

# преобразуем даты в "правильный" строчный формат    
sverka['date']=sverka.apply(date_true, axis=1)
# преобразуем даты в строчном формате в формат даты
sverka['date']=sverka['date'].apply(lambda x: datetime.strptime(x,'%d.%m.%Y'))

display(sverka.sort_values(by='date',ascending=False).head(3))

Обрезаем колонку с временем до даты.

In [ ]:
# удаляем неформатные RRN
col_rrn=sverka['rrn'].sort_values().reset_index().dropna()

col_rrn['rrn']=col_rrn['rrn'].apply(lambda x: re.findall('^\d{12}|\D|',x)[0])
col_rrn['check']=col_rrn['rrn'].apply(lambda x: len(re.findall('^\d{12}|\D|',x)[0]))

col_rrn.loc[col_rrn['check']!=12,'check']=np.nan
col_rrn=col_rrn.dropna()

# делаем из столбца строку с RRN через запятую, как надо в скрипте
ref_in=','.join(col_rrn['rrn'].astype('str'))

# вставляем строку с RRN в запрос
query_rrn = """
--вставить rrn (12ти значный) из реестра в in ()
  
select  
	  ref_number
	 ,amount
	 ,ac.account
	 ,ac.bank_name
from pycrement.payments p
left join da.pycrement.accounts ac on p.account_id = ac.id
where 2 = 2
	and p.ref_number::integer in ({})
""".format(ref_in)

Вставляем rrn и формируем текст запроса.

In [ ]:
# параметры подключения
conn_info = {'host': 'vertica.vcp.digitalaccess.ru',
             'port': 5433,
             'user': '******', # не менять!
             'password': '*******',    # не менять!
             'database': '***',
             # default throw error on invalid UTF-8 results
             'unicode_error': 'strict',
             # SSL is disabled by default
             'ssl': False,
             # using server-side prepared statements is disabled by default
             'use_prepared_statements': False,
             # connection timeout is not enabled by default
             # 5 seconds timeout for a socket operation (Establishing a TCP connection or read/write operation)
             'connection_timeout': 50000}
  
# подключение
connection = vertica_python.connect(**conn_info)
cur = connection.cursor()

# извлечение и запись
cur.execute(query_rrn)
data_rrn_mix = pd.DataFrame(cur.fetchall())
# на случай, если левак по картам (то есть их нет), как у Озона например
try: 
    data_rrn_mix.columns = ['rrn','sum','card','bank_name']
    data_rrn_mix['sum'] = data_rrn_mix['sum'].astype('float')
    data_rrn = data_rrn_mix.loc[~data_rrn_mix['rrn'].duplicated()].reset_index(drop=True)
except:
    data_rrn = pd.DataFrame(columns = ['rrn','sum','card','bank_name'])

Выгружаем из базы Vertica данные по rnn. Убираем из выгрузки дубли RRN, которые проведены по карте другого банка (такие случаи тоже замечены).

In [ ]:
sverka['card'] = sverka['card'].astype('str')
sverka['num_card']=sverka['card'].apply(lambda x:re.findall('^\d{4}',x)[0]+re.findall('\d{4}$',x)[0])
sverka['pan']=sverka['card'].apply(lambda x:re.findall('^\d{4}',x)[0])
display(sverka.head(2))

Получаем номера карт целым 8-мизначным числом.

In [ ]:
date_month = sverka.loc[len(sverka)//2,'date']

# делаем по три дня плюс\минус, потому что даты транзакций чуть могут расходиться в реестре партнёра и у нас
start_month = (first_day_of_month(date_month)-timedelta(days=3)).strftime('%Y%m%d')
finish_month = (last_day_of_month(date_month)+timedelta(days=3)).strftime('%Y%m%d')

Получаем дату начала и окончания отчётного месяца.

In [ ]:
pan=sverka['pan'].drop_duplicates().dropna().astype('str')
pan_in="display_name like '"+"%' or display_name like '".join(pan)+"%'"

if max(sverka['date'])>last_day_of_month(date_month)+timedelta(days=3):
    print("\033[37m\033[41m\033{}\033[0m".\
          format('Максимальная дата транзакций {} превышает 3-е число следующего месяца'.format(max(sverka['date']))))
else:
    query_pan="""
-- по номеру карты из Вертики----------------------------------------
			
select replace(display_name, 'x', '*') k
    ,date(debit_reportable_time) date
    ,debit_b2c sum
    ,p.ref_number rrn 
from profit.report_all_table a
inner join profit.credit b
      on a.credit_id=b.id
inner join profit.payment_system_account c
      on b.payment_system_account_id = c.id
inner join da.classified_subsite d on a.subsite_id = d.subsite_id
left join pycrement.payments p on p.id = b.plus_id
where ({}) -- номера карт
		and credit_reportable_time between '{}' and '{}'
""".format(pan_in,start_month,finish_month)

Формируем текст запроса выгрузки по номеру карты.

In [ ]:
# извлечение и запись
cur.execute(query_pan)
data_pan = pd.DataFrame(cur.fetchall())

# на случай, если ошибки по картам (то есть их нет), как у Озона например
try: 
    data_pan.columns = ['pan','date','sum','rrn']
    data_pan['sum'] = data_pan['sum'].astype('float')

    data_pan['num_card']=data_pan['pan'].apply(lambda x:re.findall('^\d{4}',x)[0]+re.findall('\d{4}$',x)[0])
    display(data_pan.sample(2))
except:
    data_pan = pd.DataFrame(columns = ['pan','date','sum','rrn','num_card'])

Выгружаем из БД транзакции за отчетный месяц по номерам карт.

In [ ]:
# добавляем к сверке суммы по RRN
sverka_merge_rrn = sverka[['bank_name','date','sum','rrn','code','card','num_card']].merge(
                                            data_rrn[['rrn','sum']], 
                                            on='rrn', 
                                            how='left',
                                            suffixes=('_sverka', '_rrn'),
                                                ).drop_duplicates().reset_index(drop=True)


# добавляем к сверке суммы по номеру карты, сумме
sverka_merge_card = sverka[['bank_name','date','sum','rrn','code','card','num_card']].merge(
                                data_pan[['num_card','sum','rrn']], 
                                on=['num_card','sum'], 
                                how='left',
                                suffixes=('_sverka', '_pan'),
                                                ).drop_duplicates().reset_index(drop=True)
sverka_merge_card.rename(columns={'sum': 'sum_pan'}, inplace=True)

# выделяем строки, которые будем засчитывать по карте (могут быть и пустые)
sverka_merge = pd.merge(sverka_merge_rrn.iloc[sverka_merge_rrn.loc[sverka_merge_rrn['sum_rrn'].isna()].index],
                        sverka_merge_card.iloc[sverka_merge_rrn.loc[sverka_merge_rrn['sum_rrn'].isna()].index][['sum_pan','rrn_pan']],
                        left_index=True, right_index=True, how='left',
                       )
# присоединяем к строкам, которые засчитываем по карте, строки по rrn
sverka_merge = pd.concat([sverka_merge,
                          sverka_merge_rrn.iloc[sverka_merge_rrn.loc[~sverka_merge_rrn['sum_rrn'].isna()].index]]).reset_index(drop=True)

# отправляем ррн из сверки, транзакций которые подтвердили по карте, в столбей code
sverka_merge.loc[~sverka_merge['rrn_pan'].isna()&~sverka_merge['rrn'].isna(),
                 'code'] = sverka_merge.loc[~sverka_merge['rrn_pan'].isna()&~sverka_merge['rrn'].isna(), 'rrn']

# перезаписываем ррн, транзакций которые подтвердили по карте, на рнн из нашей базы
sverka_merge.loc[~sverka_merge['rrn_pan'].isna(), 'rrn'] = sverka_merge.loc[~sverka_merge['rrn_pan'].isna(), 'rrn_pan']

# расставляем статус по каждой строке
sverka_merge.loc[~sverka_merge['sum_rrn'].isna()|~sverka_merge['rrn_pan'].isna(),'gap'] = 'proven'
sverka_merge.loc[sverka_merge['sum_rrn'].isna()&sverka_merge['rrn_pan'].isna(),'gap'] = 'n/a'
sverka_merge = sverka_merge.sort_values(by='gap')
display(sverka_merge.sample(2))

Присоединяем к сверке транзакции по rrn и по дата, карта (сумму не берём - может не биться в нашей базе).
По каждой строке делаем вывод принимаем или нет эту позицию.

In [ ]:
if sverka_merge['sum_sverka'].sum()/sverka['sum'].sum()>0.99:
    discrepancy_perc = str(
                        round(sverka_merge['sum_sverka'].loc[(sverka_merge['gap']=='n/a')].sum()/\
                        sverka_merge['sum_sverka'].sum()*100,
                              1)
                        )+' %'
    discrepancy_amount = str(round(sverka_merge['sum_sverka'].loc[(sverka_merge['gap']=='n/a')].sum()))+' руб'

    list_discrepancy = [discrepancy_perc,discrepancy_amount] 

    conclusion = pd.DataFrame(data=list_discrepancy, columns=['Расхождение по сумме транзакций составляет'])

    print('Расхождение по сумме транзакций составляет {} или {}.'.format(discrepancy_amount,discrepancy_perc))
else:
    conclusion = pd.DataFrame(data=['Ошибка при merge() датафреймов в Питоне'], columns=['Ошибка'])

In [ ]:
sheets = {'Сверка': sverka_merge[['bank_name','date','sum_sverka','rrn','code','card','sum_rrn','sum_pan','gap']],
          'Заключение': conclusion, 
         }
writer = pd.ExcelWriter('sverka_{}_in_{}.xlsx'.format(bank_name,date_month.strftime('%B')), engine='xlsxwriter')

for sheet_name in sheets.keys():
    sheets[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False)

writer.save()

Записываем результаты сверки в файл sverka.xlsx, лежащий в той же папке, что и файл sverka_backend.ipynb.
# Подготовка реестра для заливки в отчет по прибыли по продлениям

In [ ]:
reestr_prodleniya = sverka_merge[['bank_name','date','card','sum_sverka','code','rrn']]
reestr_prodleniya.columns = ['bank_name','transaction_date','card_num','oper_sum','auth_code', 'rrn']
reestr_prodleniya = reestr_prodleniya.fillna(0)
reestr_prodleniya['transaction_date'] = pd.to_datetime(reestr_prodleniya['transaction_date'],
                                                      format='%Y-%m-%d'
                                                      )
reestr_prodleniya['oper_sum'] = reestr_prodleniya['oper_sum'].astype('float32')
reestr_prodleniya['bank_name'] = reestr_prodleniya['bank_name'].astype('str')
reestr_prodleniya['card_num'] = reestr_prodleniya['card_num'].astype('str')
reestr_prodleniya['auth_code'] = reestr_prodleniya['auth_code'].astype('str')
reestr_prodleniya['rrn'] = reestr_prodleniya['rrn'].astype('str')

reestr_prodleniya.to_csv('reestr_prodleniya_{}_in_{}.csv'.format(bank_name,date_month.strftime('%B')), index=False)

display(reestr_prodleniya.sample(2))
reestr_prodleniya.info()


Создаём датафрейм реестра из файла сверки с форматированием колонок под тип .  
Записываем файл reestr_prodleniya.csv для импортирования в БД ClickHouse groot3.akoblov.banks_cashback_prolong.
# Подготовка реестра для заливки в отчет по прибыли от кэшбэка

In [ ]:
reestr_cashback = reestr_prodleniya
reestr_cashback.columns = ['bank_name','debit_reportable_time','CARD_NUM','debit_b2c','AUTH_CODE', 'RRN']

# определеям тип подписки
reestr_cashback.loc[reestr_cashback['debit_b2c']<399,'purchase_kind'] = 'TVOD'
reestr_cashback.loc[reestr_cashback['debit_b2c']>=399,'purchase_kind'] = 'SVOD'

# определеям длительность подписки в месяцах
reestr_cashback.loc[reestr_cashback['debit_b2c']<=500,'purchase_duration'] = 1
reestr_cashback.loc[(reestr_cashback['debit_b2c']<1000)&(reestr_cashback['debit_b2c']>500),'purchase_duration'] = 3
reestr_cashback.loc[(1000<=reestr_cashback['debit_b2c'])&(1999>reestr_cashback['debit_b2c']),'purchase_duration'] = 6
reestr_cashback.loc[reestr_cashback['debit_b2c']>=1999,'purchase_duration'] = 12

# определяем начало и конец месяца для каждой позиции
reestr_cashback['first_day_of_month'] = reestr_cashback['debit_reportable_time'].apply(lambda x: first_day_of_month(x))
reestr_cashback['last_day_of_month'] = reestr_cashback['debit_reportable_time'].apply(lambda x: last_day_of_month(x))

reestr_cashback=reestr_cashback.reset_index(drop=True)

display(reestr_cashback.tail(2))

Вычисляем вспомогательные данные для дальнейшего расчёта.

In [ ]:
# создаём пустой датафрейм, куда будем записывать раскладку каждой строки реестра
vyru4ka = pd.DataFrame(columns=['diff_month',
                                                         'prisnannay_vyru4ka',
                                                         'summa_pokupok',
                                                         'otlozhennaya_na4alo',
                                                         'otlozhennaya_konec',
                                                        ]
                     )

# перебираем строки реестра
for j in range(len(reestr_cashback)):
    a=reestr_cashback.iloc[j]['purchase_duration'].astype('int')
    b=reestr_cashback.iloc[j]['first_day_of_month']
    c=(reestr_cashback.iloc[j]['last_day_of_month']-reestr_cashback.iloc[j]['debit_reportable_time']).days+1
    d=(reestr_cashback.iloc[j]['last_day_of_month']-reestr_cashback.iloc[j]['first_day_of_month']).days+1
    e=reestr_cashback.iloc[j]['debit_b2c']/reestr_cashback.iloc[j]['purchase_duration']
    list_z = [b]
    list_prisnannay_vyru4ka = [c/d*e]
    list_summa_pokupok = [reestr_cashback.iloc[j]['debit_b2c']]
    list_otlozhennaya_na4alo = [0]
    list_otlozhennaya_konec = [list_otlozhennaya_na4alo[0]+list_summa_pokupok[0]-list_prisnannay_vyru4ka[0]]
    list_index = [j]
    for i in range(a):
        list_z.append(list_z[i]+relativedelta(months=1))
        list_prisnannay_vyru4ka.append(e)
        list_summa_pokupok.append(0)
        list_otlozhennaya_na4alo.append(list_otlozhennaya_konec[i])
        list_otlozhennaya_konec.append(list_otlozhennaya_na4alo[i+1]+\
                                       list_summa_pokupok[i+1]-list_prisnannay_vyru4ka[i+1])
        list_index.append(list_index[0])
    list_prisnannay_vyru4ka[-1]=reestr_cashback.iloc[j]['debit_b2c']-sum(list_prisnannay_vyru4ka[0:-1])
    list_otlozhennaya_konec[-1]=0

    cv = pd.DataFrame(list(zip(list_z,
                               list_prisnannay_vyru4ka,
                               list_summa_pokupok,
                               list_otlozhennaya_na4alo,
                               list_otlozhennaya_konec,
                               )
                          ), index=list_index, columns=['diff_month',
                                                         'prisnannay_vyru4ka',
                                                         'summa_pokupok',
                                                         'otlozhennaya_na4alo',
                                                         'otlozhennaya_konec',
                                                        ]
                     )
    
    vyru4ka = pd.concat([vyru4ka,cv])    
    
reestr_cashback_full = pd.merge(reestr_cashback, vyru4ka, left_index=True, right_index=True, how='left')

display(reestr_cashback_full.sample(2))

Циклом проходим по каждой строке, получая на выходе датафрейм с четырьмя финансовыми показателями.

In [ ]:
reestr_cashback_out = reestr_cashback_full

# заполняем недостающие данные
reestr_cashback_out['PARTNER_ID'] = reestr_cashback_out['AUTH_CODE']

# отбираем нужные колонки
reestr_cashback_out = reestr_cashback_out[['bank_name','purchase_kind','debit_b2c','purchase_duration',
                                             'debit_reportable_time','RRN','AUTH_CODE','CARD_NUM',
                                             'PARTNER_ID','diff_month','otlozhennaya_na4alo',
                                             'summa_pokupok','prisnannay_vyru4ka','otlozhennaya_konec']]

# переименовываем колонки
reestr_cashback_out.columns = ['bank_name','purchase_kind','debit_b2c','purchase_duration',
                                             'debit_reportable_time','rrn','auth_code','card_num',
                                             'partner_id','report_month','rev_part_start','amount',
                                             'rev_part','rev_part_finish']

# меняем формат данных в каждой колонке
reestr_cashback_out = reestr_cashback_out.fillna(0)
reestr_cashback_out['debit_reportable_time'] = pd.to_datetime(reestr_cashback_out['debit_reportable_time'],
                                                      format='%Y-%m-%d'
                                                      )
reestr_cashback_out['report_month'] = pd.to_datetime(reestr_cashback_out['report_month'],
                                                      format='%Y-%m-%d'
                                                      )

reestr_cashback_out['debit_b2c'] = reestr_cashback_out['debit_b2c'].astype('float32')
reestr_cashback_out['rev_part_start'] = reestr_cashback_out['rev_part_start'].astype('float32')
reestr_cashback_out['amount'] = reestr_cashback_out['amount'].astype('float32')
reestr_cashback_out['rev_part'] = reestr_cashback_out['rev_part'].astype('float32')
reestr_cashback_out['rev_part_finish'] = reestr_cashback_out['rev_part_finish'].astype('float32')
reestr_cashback_out['purchase_duration'] = reestr_cashback_out['purchase_duration'].astype('int32')

reestr_cashback_out['bank_name'] = reestr_cashback_out['bank_name'].astype('str')
reestr_cashback_out['purchase_kind'] = reestr_cashback_out['purchase_kind'].astype('str')
reestr_cashback_out['rrn'] = reestr_cashback_out['rrn'].astype('str')
reestr_cashback_out['auth_code'] = reestr_cashback_out['auth_code'].astype('str')
reestr_cashback_out['card_num'] = reestr_cashback_out['card_num'].astype('str')
reestr_cashback_out['partner_id'] = reestr_cashback_out['partner_id'].astype('str')

# записываем результат в csv, отбрасывая нули
reestr_cashback_out = reestr_cashback_out.query('rev_part_start!=0 or amount!=0 or rev_part!=0 or rev_part_finish!=0')
reestr_cashback_out.to_csv('reestr_cashback_{}_in_{}.csv'.format(bank_name,date_month.strftime('%B')), index=False)

display(reestr_cashback_out.sample(2))
reestr_cashback_out.info()

# Вывод

In [ ]:
print("\033[37m\033[41m\033{}".format('Выполнение программы завершено успешно'))